# keras训练和评估

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model中训练和评估的函数

fit:训练

evaluate：评估

predict：预测

In [6]:
# 样例如下
inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, activation="softmax", name="predictions")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [7]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [8]:
# 转换数据，并划分训练、验证、测试集合
# Preprocess the data (these are NumPy arrays)
x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Reserve 10,000 samples for validation
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

In [9]:
# 定义训练过程配置
model.compile(optimizer=keras.optimizers.Adam(1e-4),loss=keras.losses.SparseCategoricalCrossentropy(),metrics=[keras.metrics.SparseCategoricalAccuracy()])


In [10]:
print("Fit model on training data")
# 定义epoch和batch-size
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=2,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(x_val, y_val),
)

Fit model on training data
Train on 50000 samples, validate on 10000 samples
Epoch 1/2
50000/50000 [==============================] - 3s 56us/sample - loss: 0.9824 - sparse_categorical_accuracy: 0.7394 - val_loss: 0.3978 - val_sparse_categorical_accuracy: 0.8978
Epoch 2/2
50000/50000 [==============================] - 2s 33us/sample - loss: 0.3661 - sparse_categorical_accuracy: 0.9000 - val_loss: 0.2913 - val_sparse_categorical_accuracy: 0.9170


In [11]:
history.history

{'loss': [0.9823776105308533, 0.3661482168340683],
 'sparse_categorical_accuracy': [0.73936, 0.90004],
 'val_loss': [0.3978313138961792, 0.29129388489723207],
 'val_sparse_categorical_accuracy': [0.8978, 0.917]}

In [12]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=64,verbose=2)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(x_test[:3])
print("predictions shape:", predictions.shape)

Evaluate on test data
10000/1 - 0s - loss: 0.2285 - sparse_categorical_accuracy: 0.9174
test loss, test acc: [0.2982235023736954, 0.9174]
Generate predictions for 3 samples
predictions shape: (3, 10)


## 关于compile中许多内置的方法

- optimizer
    - SGD()
    - RMSprop()
    - Adam()
    - etc.
- Losses:
    - MeanSquaredError()
    - KLDivergence()
    - CosineSimilarity()
    - etc.
- Metrics:
    - AUC()
    - Precision()
    - Recall()
    - etc.
    
如果要自定义loss，需要继承tf.keras.losses.Loss类，此时需要重写__init__ and call(self,y_true,y_pred)

示例如下

In [9]:
class CustomMSE(keras.losses.Loss):
    def __init__(self, regularization_factor=0.1, name="custom_mse"):
        super().__init__(name=name)
        self.regularization_factor = regularization_factor

    def call(self, y_true, y_pred):
        mse = tf.math.reduce_mean(tf.square(y_true - y_pred))
        reg = tf.math.reduce_mean(tf.square(0.5 - y_pred))
        return mse + reg * self.regularization_factor

如果要自定义metrics，需要继承tf.keras.metrics.Metric，需要实现4中方法：
- __init__(self):为了你的metric目标创建中间参数
- update_state(self, y_true, y_pred, sample_weight=None)：利用真实值和预测值更新中间值
- result(self):利用中间值得出最终结果
- reset_states(self):重新初始化中间状态

示例如下：

In [14]:
class CategoricalTruePositives(keras.metrics.Metric):
    def __init__(self, name="categorical_true_positives", **kwargs):
        super(CategoricalTruePositives, self).__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name="ctp", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.reshape(tf.argmax(y_pred, axis=1), shape=(-1, 1))
        values = tf.cast(y_true, "int32") == tf.cast(y_pred, "int32")
        values = tf.cast(values, "float32")
        if sample_weight is not None:
            sample_weight = tf.cast(sample_weight, "float32")
            values = tf.multiply(values, sample_weight)
        self.true_positives.assign_add(tf.reduce_sum(values))

    def result(self):
        return self.true_positives

    def reset_states(self):
        # The state of the metric will be reset at the start of each epoch.
        self.true_positives.assign(0.0)

In [15]:
def get_uncompiled_model():
    inputs = keras.Input(shape=(784,), name="digits")
    x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
    x = layers.Dense(64, activation="relu", name="dense_2")(x)
    outputs = layers.Dense(10, activation="softmax", name="predictions")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

def get_compiled_model():
    model = get_uncompiled_model()
    model.compile(
        optimizer="rmsprop",
        loss="sparse_categorical_crossentropy",
        metrics=["sparse_categorical_accuracy"],
    )
    return model

model = get_uncompiled_model()
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[CategoricalTruePositives()],
)
model.fit(x_train, y_train, batch_size=64, epochs=3)

Train on 50000 samples
Epoch 1/3
50000/50000 [==============================] - 2s 37us/sample - loss: 0.3393 - categorical_true_positives: 45154.0000
Epoch 2/3
50000/50000 [==============================] - 1s 28us/sample - loss: 0.1598 - categorical_true_positives: 47623.0000
Epoch 3/3
50000/50000 [==============================] - 1s 28us/sample - loss: 0.1163 - categorical_true_positives: 48272.0000


In [16]:
# 添加正则化损失
class ActivityRegulariationLayer(keras.layers.Layer):
    def call(self,inputs):
        self.add_loss(tf.math.reduce_sum(inputs) * 0.1)
        return inputs
inputs = keras.Input(shape=(784,),name='digits')
x=layers.Dense(64,activation='relu',name='dense_1')(inputs)
x=ActivityRegulariationLayer()(x)
x = layers.Dense(64,activation='relu',name='dense_2')(x)
outputs = layers.Dense(10,name='pred')(x)
model=keras.Model(inputs=inputs,outputs=outputs)
model.compile(optimizer=keras.optimizers.RMSprop(1e-3),loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True))
model.fit(x_train,y_train,batch_size=64,epochs=1)

Train on 50000 samples
50000/50000 [==============================] - 2s 37us/sample - loss: 2.4918


也可以添加对评估函数的日志记录-利用add_metric()方法


In [17]:
class MetricLoggingLayer(layers.Layer):
    def call(self,inputs):
        self.add_metric(keras.backend.std(inputs),name='std_of_activation',aggregation='mean')
        return inputs
inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)

# Insert std logging as a layer.
x = MetricLoggingLayer()(x)

x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, name="predictions")(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)
model.fit(x_train, y_train, batch_size=64, epochs=1)

Train on 50000 samples
50000/50000 [==============================] - 2s 35us/sample - loss: 0.3415 - std_of_activation: 0.9822


In [18]:
# 在创建好模型之后 也可以利用模型model的方法 add_loss add_metric
inputs = keras.Input(shape=(784,), name="digits")
x1 = layers.Dense(64, activation="relu", name="dense_1")(inputs)
x2 = layers.Dense(64, activation="relu", name="dense_2")(x1)
outputs = layers.Dense(10, name="predictions")(x2)
model = keras.Model(inputs=inputs, outputs=outputs)

model.add_loss(tf.reduce_sum(x1) * 0.1)

model.add_metric(keras.backend.std(x1), name="std_of_activation", aggregation="mean")

model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)
model.fit(x_train, y_train, batch_size=64, epochs=1)

Train on 50000 samples
50000/50000 [==============================] - 2s 40us/sample - loss: 2.5361 - std_of_activation: 0.0020


In [19]:
# 利用验证集
model = get_compiled_model()
model.fit(x_train, y_train, batch_size=64, validation_split=0.2, epochs=1)

Train on 40000 samples, validate on 10000 samples
40000/40000 [==============================] - 2s 52us/sample - loss: 0.3632 - sparse_categorical_accuracy: 0.8988 - val_loss: 0.2199 - val_sparse_categorical_accuracy: 0.9334


In [20]:
model = get_compiled_model()

# First, let's create a training Dataset instance.
# For the sake of our example, we'll use the same MNIST data as before.
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# Shuffle and slice the dataset.
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(128)

# Now we get a test dataset.
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.batch(128)

# Since the dataset already takes care of batching,
# we don't pass a `batch_size` argument.
model.fit(train_dataset, epochs=3)

# You can also evaluate or predict on a dataset.
print("Evaluate")
result = model.evaluate(test_dataset)
dict(zip(model.metrics_names, result))

Epoch 1/3
391/391 [==============================] - 3s 8ms/step - loss: 0.4031 - sparse_categorical_accuracy: 0.8893
Epoch 2/3
391/391 [==============================] - 1s 3ms/step - loss: 0.1971 - sparse_categorical_accuracy: 0.9434
Epoch 3/3
391/391 [==============================] - 1s 3ms/step - loss: 0.1470 - sparse_categorical_accuracy: 0.9570
Evaluate
79/79 [==============================] - 0s 2ms/step - loss: 0.1388 - sparse_categorical_accuracy: 0.9602


{'loss': 0.13879615733329254, 'sparse_categorical_accuracy': 0.9602}

利用dataset的验证集

In [24]:
model = get_compiled_model()

# Prepare the training dataset
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(128)

# Prepare the validation dataset
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(64)

model.fit(train_dataset, epochs=3, validation_data=val_dataset)

Epoch 1/3
391/391 [==============================] - 3s 7ms/step - loss: 0.3984 - sparse_categorical_accuracy: 0.8893 - val_loss: 0.0000e+00 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 2/3
391/391 [==============================] - 1s 3ms/step - loss: 0.1884 - sparse_categorical_accuracy: 0.9448 - val_loss: 0.1576 - val_sparse_categorical_accuracy: 0.9547
Epoch 3/3
391/391 [==============================] - 1s 3ms/step - loss: 0.1397 - sparse_categorical_accuracy: 0.9582 - val_loss: 0.1359 - val_sparse_categorical_accuracy: 0.9579


## 支持其他的输入格式

目前常用的输入格式有
- Numpy
- Dataset
- Sequence

前两个上面已经试验过，而利用Sequence，需要继承类 keras.util.Sequence，且必须实现 __getitem__ 、__len__方法，__getitem__ 方法必须返回一个完整的batch，如果你想在每次epoch之间改变数据，那么你还需要实现on_epoch_end 方法

In [26]:
# 对于Sequence的实践
class CIFAR10Sequence(keras.utils.Sequence):
    def __init__(self, filenames, labels, batch_size):
        self.filenames, self.labels = filenames, labels
        self.batch_size = batch_size

    def __len__(self):
        # 返回总batch数目
        return int(np.ceil(len(self.filenames) / float(self.batch_size)))

    def __getitem__(self, idx):
        # 利用idx值来作为step数，得到一个batch的 x,y
        batch_x = self.filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        return np.array([
            resize(imread(filename), (200, 200))
               for filename in batch_x]), np.array(batch_y)
# 和Dataset类似的使用方法

## 利用样本权重和类别权重

默认情况下，数据集中的样本权重由他的频数来确定，有两种方法来对数据权重进行赋值
- 类别权重
- 样本权重

### 类别权重
在fit()的时候传入 class_weight ，这是一个dict格式，定义每个类别的权重值，类型为double。一般来说对于数量较少的类赋予较大的权重，有助于缓解样本不平衡训练,例如如下格式
```python
import numpy as np

class_weight = {
    0: 1.0,
    1: 1.0,
    2: 1.0,
    3: 1.0,
    4: 1.0,
    # Set weight "2" for class "5",
    # making this class 2x more important
    5: 2.0,
    6: 1.0,
    7: 1.0,
    8: 1.0,
    9: 1.0,
}

print("Fit with class weight")
model = get_compiled_model()
model.fit(x_train, y_train, class_weight=class_weight, batch_size=64, epochs=1)
```

### 样本权重
当没建立分类模型，可以使用样本权重，此时对于tf.keras而言，有两种情况：
- 输入数据为Numpy，此时fit()使用sample_weight参数 
- 输入数据为Dataset或者其他迭代器类型：Yield（input_batch,label_batch,sample_weight_batch） tuples
样本权重一般为数组形式，指定每个数据在batch的权重值，同样用在样本不平衡的场景下，少数量的样本，权重更大。示例如下代码

```python
sample_weight = np.ones(shape=(len(y_train),))
sample_weight[y_train == 5] = 2.0

print("Fit with sample weight")
model = get_compiled_model()
model.fit(x_train, y_train, sample_weight=sample_weight, batch_size=64, epochs=1)
```
下面是对于Dataset的版本

```python
sample_weight = np.ones(shape=(len(y_train),))
sample_weight[y_train == 5] = 2.0

# Create a Dataset that includes sample weights
# (3rd element in the return tuple).
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train, sample_weight))

# Shuffle and slice the dataset.
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

model = get_compiled_model()
model.fit(train_dataset, epochs=1)
```

## 多输入和多输出的模型创建
以上的模型均为单输入和单输出的模型，对于多输入和输出模型也类似

In [27]:
# 两个输入
image_input = keras.Input(shape=(32, 32, 3), name="img_input")
timeseries_input = keras.Input(shape=(None, 10), name="ts_input")

x1 = layers.Conv2D(3, 3)(image_input)
x1 = layers.GlobalMaxPooling2D()(x1)

x2 = layers.Conv1D(3, 3)(timeseries_input)
x2 = layers.GlobalMaxPooling1D()(x2)

x = layers.concatenate([x1, x2])

# 两个输出
score_output = layers.Dense(1, name="score_output")(x)
class_output = layers.Dense(5, activation="softmax", name="class_output")(x)

model = keras.Model(
    inputs=[image_input, timeseries_input], outputs=[score_output, class_output]
)

In [29]:
# 在complie种需要注意，有多种方法可以进行设置
# 1.loss按照数组设置，分别对应添加的两个不同的输出，如果指定一个loss，那么对于不同输出会应用一样的loss函数
model.compile(optimizer=keras.optimizers.RMSprop(1e-3),loss=[keras.losses.MeanSquaredError(),keras.losses.CategoricalCrossentropy()])
# 2.对于metrics
model.compile(
    optimizer=keras.optimizers.RMSprop(),
    loss=[keras.losses.MeanSquaredError(),keras.losses.CategoricalCrossentropy()],
    metrics=[
        [keras.metrics.MeanAbsolutePercentageError(),
         keras.metrics.MeanAbsoluteError()
        ],
        [keras.metrics.CategoricalAccuracy()]
    ]
)
# 3.类似的  我们还可以指定名称，利用[dict]这种方式来设置
model.compile(
    optimizer=keras.optimizers.RMSprop(),
    loss={
        'score_output':keras.losses.MeanSquaredError(),
        'class_output':keras.losses.CategoricalCrossentropy()
    },
    metrics={
        'score_output':[
            keras.metrics.MeanAbsolutePercentageError(),
            keras.metrics.MeanAbsoluteError()
        ],
        'class_output':[keras.metrics.CategoricalAccuracy()]
    }
)
# 4.同样也可以对不同的输出损失进行权重指定
model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss={
        "score_output": keras.losses.MeanSquaredError(),
        "class_output": keras.losses.CategoricalCrossentropy()
    },
    metrics={
        "score_output": [
            keras.metrics.MeanAbsolutePercentageError(),
            keras.metrics.MeanAbsoluteError(),
        ],
        "class_output": [keras.metrics.CategoricalAccuracy()]
    },
    loss_weights={"score_output": 2.0, "class_output": 1.0},
)
# 5.也可以不选择损失函数 ，仅把输出结果显示不用来训练
# List loss version
model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss=[None, keras.losses.CategoricalCrossentropy()],
)

# Or dict loss version
model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss={"class_output": keras.losses.CategoricalCrossentropy()},
)

In [31]:
# 在训练过程中也是要指定名称来对输入输出数据集指定
import numpy as np
model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss=[keras.losses.MeanSquaredError(), keras.losses.CategoricalCrossentropy()],
)

# Generate dummy NumPy data
img_data = np.random.random_sample(size=(100, 32, 32, 3))
ts_data = np.random.random_sample(size=(100, 20, 10))
score_targets = np.random.random_sample(size=(100, 1))
class_targets = np.random.random_sample(size=(100, 5))

# Fit on lists
model.fit([img_data, ts_data], [score_targets, class_targets], batch_size=32, epochs=1)

# Alternatively, fit on dicts
model.fit(
    {"img_input": img_data, "ts_input": ts_data},
    {"score_output": score_targets, "class_output": class_targets},
    batch_size=32,
    epochs=1,
)

Train on 100 samples
100/100 [==============================] - 2s 16ms/sample - loss: 6.7461 - score_output_loss: 2.1003 - class_output_loss: 4.9883
Train on 100 samples
100/100 [==============================] - 0s 366us/sample - loss: 5.8471 - score_output_loss: 1.2582 - class_output_loss: 4.6432


In [32]:
# 当传入的数据为Dataset格式，那么需要在生成Dataset时进行名称指定
train_dataset = tf.data.Dataset.from_tensor_slices(
    (
        {"img_input": img_data, "ts_input": ts_data},
        {"score_output": score_targets, "class_output": class_targets},
    )
)
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

model.fit(train_dataset, epochs=1)

2/2 [==============================] - 1s 255ms/step - loss: 5.4599 - score_output_loss: 0.9415 - class_output_loss: 4.5185


## 利用callbacks
callbakcs一般在keras种用来为不同的训练目标来设定的，一般有以下实现的功能
1. 在训练中 进行可视化
2. checkpoint
3. 改变学习速率
4. 在训练中对层做fune-tuning
5. 发送Email或其他一些信息
6. etc.

callbacks 一般在fit()使用，例如如下 常用的早停机制

```python
model = get_compiled_model()

callbacks = [
    keras.callbacks.EarlyStopping(
        # Stop training when `val_loss` is no longer improving
        monitor="val_loss",
        # "no longer improving" being defined as "no better than 1e-2 less"
        min_delta=1e-2,
        # "no longer improving" being further defined as "for at least 2 epochs"
        patience=2,
        verbose=1,
    )
]
model.fit(
    x_train,
    y_train,
    epochs=20,
    batch_size=64,
    callbacks=callbacks,
    validation_split=0.2,
)
```

### 编写自己的callback
需要继承 keras.callbacks.Callback类，在fit、evaluate、predict种均可使用，在继承类之后按需要实现以下方法，实现callback
- on_(train|test|predict)_begin(self, logs=None) 在训练/验证/测试 开始时
- on_(train|test|predict)_end(self, logs=None) 在训练/验证/测试 结束后
- on_(train|test|predict)_batch_begin(self, batch, logs=None) 在训练/验证/测试 batch开始
- on_(train|test|predict)_batch_end(self, batch, logs=None) 在训练/验证/测试 batch结束
- on_epoch_begin(self, epoch, logs=None) 在每一个epoch开始 仅train生效
- on_epoch_end(self, epoch, logs=None)  在每一个epoch结束 仅train生效

其中logs是一个dict，里面包含了每个步骤下的loss，step中为trainloss，在epoch结束时有loss 和设定的其他指标入 metrics等

### Checkpoint
当使用大量数据进行模型训练时，应该在一定的频次间隔保存训练好的模型，最简单的就是利用 keras.callbacks.ModelCheckpoint 来设定


In [33]:
model = get_compiled_model()

callbacks = [
    keras.callbacks.ModelCheckpoint(
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        # The saved model name will include the current epoch.
        filepath="mymodel_{epoch}",
        save_best_only=True,  # Only save a model if `val_loss` has improved.
        monitor="val_loss",
        verbose=1,
    )
]
model.fit(
    x_train, y_train, epochs=2, batch_size=64, callbacks=callbacks, validation_split=0.2
)

Train on 40000 samples, validate on 10000 samples
Epoch 1/2
38464/40000 [===========================>..] - ETA: 0s - loss: 0.3804 - sparse_categorical_accuracy: 0.8928
Epoch 00001: val_loss improved from inf to 0.23963, saving model to mymodel_1
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: mymodel_1\assets
40000/40000 [==============================] - 3s 63us/sample - loss: 0.3741 - sparse_categorical_accuracy: 0.8946 - val_loss: 0.2396 - val_sparse_categorical_accuracy: 0.9270
Epoch 2/2
39872/40000 [============================>.] - ETA: 0s - loss: 0.1778 - sparse_categorical_accuracy: 0.9480
Epoch 00002: val_loss improved from 0.23963 to 0.20471, saving model to mymodel_2
INFO:tensorflow:Assets written to: mymodel_2\assets
40000/40000 [==============================] - 2s 48us/sample - loss: 0.1778 - sparse_categorical_accuracy: 0.9480 - val_loss: 0.2047 - val_sparse_categorical_accuracy: 0.9375


## 在训练中保存模型并在停止时恢复上次训练的模型

In [ ]:
import os

# Prepare a directory to store all the checkpoints.
checkpoint_dir = ".\\ckpt"
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)


def make_or_restore_model():
    # Either restore the latest model, or create a fresh one
    # if there is no checkpoint available.
    checkpoints = [checkpoint_dir + "\\" + name for name in os.listdir(checkpoint_dir)]
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=os.path.getctime)
        print("Restoring from", latest_checkpoint)
        return keras.models.load_model(latest_checkpoint)
    print("Creating a new model")
    return get_compiled_model()


model = make_or_restore_model()
callbacks = [
    # This callback saves a SavedModel every 100 batches.
    # We include the training loss in the saved model name.
    keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_dir + "\\ckpt-loss={loss:.2f}", save_freq=100
    )
]
model.fit(x_train, y_train, epochs=1, callbacks=callbacks)

## 运用学习率schedules

In [37]:
initial_learning_rate = 0.1
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,decay_steps=100000,decay_rate=0.96,staircase=True
)
# schedules 包括了 ExponentialDecay、PiecewiseConstantDecay、PolynomialDecay、InverseTimeDecay
optimizer = keras.optimizers.RMSprop(learning_rate=lr_schedule)

## TensorBoard可视化

有两种方式运行
第一种：按照callback来做
第二种：自己编写每一步train_step,在tf.GradientTape()外调用，记录每一步的loss或者其他指标

In [38]:
# 第一种
log_dir = '.\\logs\\fit\\'
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1)
model.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test),callbacks=[tensorboard_callback])

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 6s 122us/sample - loss: 0.1457 - sparse_categorical_accuracy: 0.9569 - val_loss: 0.1188 - val_sparse_categorical_accuracy: 0.9653
Epoch 2/5
50000/50000 [==============================] - 6s 111us/sample - loss: 0.1072 - sparse_categorical_accuracy: 0.9688 - val_loss: 0.1188 - val_sparse_categorical_accuracy: 0.9647
Epoch 3/5
50000/50000 [==============================] - 6s 111us/sample - loss: 0.0887 - sparse_categorical_accuracy: 0.9743 - val_loss: 0.1086 - val_sparse_categorical_accuracy: 0.9706
Epoch 4/5
50000/50000 [==============================] - 6s 116us/sample - loss: 0.0763 - sparse_categorical_accuracy: 0.9784 - val_loss: 0.1102 - val_sparse_categorical_accuracy: 0.9716
Epoch 5/5
50000/50000 [==============================] - 5s 106us/sample - loss: 0.0661 - sparse_categorical_accuracy: 0.9813 - val_loss: 0.1194 - val_sparse_categorical_accuracy: 0.9694


In [39]:
# 第二种
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

In [40]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
# Define our metrics
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

def train_step(model, optimizer, x_train, y_train):
  with tf.GradientTape() as tape:
    predictions = model(x_train, training=True)
    loss = loss_object(y_train, predictions)
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  train_loss(loss)
  train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
  predictions = model(x_test)
  loss = loss_object(y_test, predictions)

  test_loss(loss)
  test_accuracy(y_test, predictions)

In [42]:
import datetime

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [44]:
def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])
model = create_model() # reset our model

EPOCHS = 5

for epoch in range(EPOCHS):
  for (x_train, y_train) in train_dataset:
    train_step(model, optimizer, x_train, y_train)
  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

  for (x_test, y_test) in test_dataset:
    test_step(model, x_test, y_test)
  with test_summary_writer.as_default():
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(), 
                         train_accuracy.result()*100,
                         test_loss.result(), 
                         test_accuracy.result()*100))

  # Reset metrics every epoch
  train_loss.reset_states()
  test_loss.reset_states()
  train_accuracy.reset_states()
  test_accuracy.reset_states()

Epoch 1, Loss: 0.268586665391922, Accuracy: 92.25399780273438, Test Loss: 0.1262025088071823, Test Accuracy: 96.31999969482422
Epoch 2, Loss: 0.11531289666891098, Accuracy: 96.552001953125, Test Loss: 0.09362193197011948, Test Accuracy: 97.23999786376953
Epoch 3, Loss: 0.07867169380187988, Accuracy: 97.58399963378906, Test Loss: 0.0755162239074707, Test Accuracy: 97.61000061035156
Epoch 4, Loss: 0.058402884751558304, Accuracy: 98.21599578857422, Test Loss: 0.07108774036169052, Test Accuracy: 97.86000061035156
Epoch 5, Loss: 0.045410677790641785, Accuracy: 98.51599884033203, Test Loss: 0.07189565151929855, Test Accuracy: 97.81999969482422


# END